### Data Engineering (Data Preparation)

The second phase of the CRISP-ML(Q) process model aims to prepare data for the following modeling phase. Data selection, data cleaning, feature engineering, and data standardization tasks are performed during this phase.
We identify valuable and necessary features for future model training by using either filter methods, wrapper methods, or embedded methods for data selection. Furthermore, we select data by discarding samples that do not satisfy data quality requirements. At this point, we also might tackle the problem of unbalanced classes by applying over-sampling or under-sampling strategies.
The data cleaning task implies that we perform error detection and error correction steps for the available data. Adding unit testing for data will mitigate the risk of error propagation to the next phase. Depending on the machine learning task, we might need to perform feature engineering and data augmentation activities. For example, such methods include one-hot encoding, clustering, or discretization of continuous attributes.
The data standardization task denotes the process of unifying the ML tools’ input data to avoid the risk of erroneous data. Finally, the normalization task will mitigate the risk of bias to features on larger scales. We build data and input data transformation pipelines for data pre-processing and feature creation to ensure the ML application’s reproducibility during this phase.

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

In [2]:
def readFile(name):
    df = pd.read_json(name, encoding = 'ISO-8859-1')
    print(df.head())
    return df

In [3]:
vehicles_sales = readFile('../Data/Maestria_sls_Dummy_S.json')

# Count the number of records (rows)
num_records = vehicles_sales.shape[0]
print("Number of records:", num_records)

                    DL_NM    BR      PR   TS              LINE    TYPE  TOT
0  Dimotors Coatzacoalcos  FORD  202401  MTD  RANGER SILVERTON  TRUCKS    1
1  Dimotors Coatzacoalcos  FORD  202401  MTD  RANGER SILVERTON  TRUCKS    3
2  Dimotors Coatzacoalcos  FORD  202401  MTD         LOBO CREW  TRUCKS    2
3  Dimotors Coatzacoalcos  FORD  202401  MTD          MAVERICK  TRUCKS    4
4  Dimotors Coatzacoalcos  FORD  202401  MTD  RANGER SILVERTON  TRUCKS    7
Number of records: 42043


In [4]:
# Rename columns
vehicles_sales.rename(columns={'DL_NM': 'Dealer_Name', 'BR': 'Brand', 'PR': 'Period', 'TS': 'Time', 
                               'LINE': 'Vehicle_Line', 'TYPE': 'Vehicle_Type', 'TOT': 'Total_Sales'}, inplace=True)

vehicles_sales.head()

,Dealer_Name,Brand,Period,Time,Vehicle_Line,Vehicle_Type,Total_Sales
0,Dimotors Coatzacoalcos,FORD,202401,MTD,RANGER SILVERTON,TRUCKS,1
1,Dimotors Coatzacoalcos,FORD,202401,MTD,RANGER SILVERTON,TRUCKS,3
2,Dimotors Coatzacoalcos,FORD,202401,MTD,LOBO CREW,TRUCKS,2
3,Dimotors Coatzacoalcos,FORD,202401,MTD,MAVERICK,TRUCKS,4
4,Dimotors Coatzacoalcos,FORD,202401,MTD,RANGER SILVERTON,TRUCKS,7


In [5]:
# Convert "Dealer_Name" column to uppercase
vehicles_sales['Dealer_Name'] = vehicles_sales['Dealer_Name'].str.upper()
vehicles_sales['Brand'] = vehicles_sales['Brand'].str.upper()

# Remove duplicate items
vehicles_sales.drop_duplicates(inplace=True)

# Count the number of records (rows)
num_records = vehicles_sales.shape[0]
print("Number of records:", num_records)


Number of records: 29244


In [6]:
# Remove rows with blank or null values in "Total_Sales" column
vehicles_sales = vehicles_sales.dropna(subset=['Total_Sales'])

# Check for consistency in data values (Example: Check if 'Total_Sales' is non-negative)
vehicles_sales_less = vehicles_sales[vehicles_sales['Total_Sales'] >= 0]

# Count the number of records (rows)
num_records = vehicles_sales.shape[0]
print("Number of records:", num_records)


Number of records: 29244


In [7]:
vehicles_sales.head()

,Dealer_Name,Brand,Period,Time,Vehicle_Line,Vehicle_Type,Total_Sales
0,DIMOTORS COATZACOALCOS,FORD,202401,MTD,RANGER SILVERTON,TRUCKS,1
1,DIMOTORS COATZACOALCOS,FORD,202401,MTD,RANGER SILVERTON,TRUCKS,3
2,DIMOTORS COATZACOALCOS,FORD,202401,MTD,LOBO CREW,TRUCKS,2
3,DIMOTORS COATZACOALCOS,FORD,202401,MTD,MAVERICK,TRUCKS,4
4,DIMOTORS COATZACOALCOS,FORD,202401,MTD,RANGER SILVERTON,TRUCKS,7


In [8]:
# Split the "Period" column into "Year" and "Month"
vehicles_sales['Year'] = vehicles_sales['Period'] // 100  # Extract the year
vehicles_sales['Month'] = vehicles_sales['Period'] % 100  # Extract the month

# Drop the "Period" column
vehicles_sales.drop(columns=['Period'], inplace=True)

# Map numerical month values to month names
month_map = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}
vehicles_sales['Month'] = vehicles_sales['Month'].map(month_map)

# Move the "Year" and "Month" columns to the 3rd position
vehicles_sales.insert(2, 'Year', vehicles_sales.pop('Year'))
vehicles_sales.insert(3, 'Month', vehicles_sales.pop('Month'))

# Display the DataFrame with the new "Year" and "Month" columns
vehicles_sales.head()

,Dealer_Name,Brand,Year,Month,Time,Vehicle_Line,Vehicle_Type,Total_Sales
0,DIMOTORS COATZACOALCOS,FORD,2024,January,MTD,RANGER SILVERTON,TRUCKS,1
1,DIMOTORS COATZACOALCOS,FORD,2024,January,MTD,RANGER SILVERTON,TRUCKS,3
2,DIMOTORS COATZACOALCOS,FORD,2024,January,MTD,LOBO CREW,TRUCKS,2
3,DIMOTORS COATZACOALCOS,FORD,2024,January,MTD,MAVERICK,TRUCKS,4
4,DIMOTORS COATZACOALCOS,FORD,2024,January,MTD,RANGER SILVERTON,TRUCKS,7


In [9]:
# Group by specified columns and sum "Total_Sales"
grouped_sales = vehicles_sales.groupby(['Dealer_Name', 'Brand', 'Year', 'Month', 'Time', 'Vehicle_Line', 'Vehicle_Type'])['Total_Sales'].sum().reset_index()
grouped_sales = grouped_sales.sort_values(by=['Month', 'Dealer_Name', 'Total_Sales'])

# Sort the DataFrame by month in the correct order
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
grouped_sales['Month'] = pd.Categorical(grouped_sales['Month'], categories=month_order, ordered=True)
grouped_sales = grouped_sales.sort_values(by=['Dealer_Name', 'Brand', 'Year', 'Month', 'Time'])

grouped_sales.head()

,Dealer_Name,Brand,Year,Month,Time,Vehicle_Line,Vehicle_Type,Total_Sales
21,ACASA PERINORTE,FORD,2024,January,MTD,F-450,TRUCKS,1
22,ACASA PERINORTE,FORD,2024,January,MTD,FORD BRONCO,OUTFITTERS,2
23,ACASA PERINORTE,FORD,2024,January,MTD,LOBO CREW,TRUCKS,6
19,ACASA PERINORTE,FORD,2024,January,MTD,E-TRANSIT,TRUCKS,25
26,ACASA PERINORTE,FORD,2024,January,MTD,TRANSIT COURIER,TRUCKS,28


In [10]:
# Count the number of records (rows)
num_records = grouped_sales.shape[0]
print("Number of records:", num_records)



Number of records: 4481


In [11]:
grouped_sales_quarterly = grouped_sales.copy()
grouped_sales_quarterly['Time'] = 'QTR'

# Map 'Month' column to represent quarters
quarter_map = {
    'January': 'Q1',
    'February': 'Q1',
    'March': 'Q1',
    'April': 'Q2',
    'May': 'Q2',
    'June': 'Q2',
    'July': 'Q3',
    'August': 'Q3',
    'September': 'Q3',
    'October': 'Q4',
    'November': 'Q4',
    'December': 'Q4'
}
grouped_sales_quarterly['Month'] = grouped_sales_quarterly['Month'].map(quarter_map)

# Group by specified columns and sum 'Total_Sales'
grouped_sales_quarterly = grouped_sales_quarterly.groupby(['Dealer_Name', 'Brand', 'Year', 'Month', 'Time', 'Vehicle_Line', 'Vehicle_Type'])['Total_Sales'].sum().reset_index()

# Sort the DataFrame if needed
grouped_sales_quarterly = grouped_sales_quarterly.sort_values(by=['Dealer_Name', 'Brand', 'Year', 'Month', 'Time'])



# Display the resulting DataFrame
print("Number of records:", grouped_sales_quarterly.shape[0])
print(grouped_sales_quarterly.head())


Number of records: 2790
       Dealer_Name Brand  Year Month Time    Vehicle_Line Vehicle_Type  \
0  ACASA PERINORTE  FORD  2024    Q1  QTR    BRONCO SPORT   OUTFITTERS   
1  ACASA PERINORTE  FORD  2024    Q1  QTR       E-TRANSIT       TRUCKS   
2  ACASA PERINORTE  FORD  2024    Q1  QTR  ESCAPE NA FHEV   OUTFITTERS   
3  ACASA PERINORTE  FORD  2024    Q1  QTR       EXPED MAX   OUTFITTERS   
4  ACASA PERINORTE  FORD  2024    Q1  QTR           F-150       TRUCKS   

   Total_Sales  
0           77  
1           25  
2          192  
3           57  
4           34  


In [12]:
# Custom search criteria
dealer_name_criteria = 'ZAPATA'
months_criteria = ['Q1']
vehicle_line_criteria = 'BRONCO SPORT'

# Perform the search
custom_search_result = grouped_sales_quarterly[
    (grouped_sales_quarterly['Dealer_Name'] == dealer_name_criteria) &
    (grouped_sales_quarterly['Month'].isin(months_criteria)) &
    (grouped_sales_quarterly['Vehicle_Line'] == vehicle_line_criteria)
]

# Display the search result
print(custom_search_result)


     Dealer_Name Brand  Year Month Time  Vehicle_Line Vehicle_Type  \
2691      ZAPATA  FORD  2024    Q1  QTR  BRONCO SPORT   OUTFITTERS   

      Total_Sales  
2691          227  


In [13]:
# Custom search criteria to validate Quarter Calculation
months_criteria = ['January', 'February','March']

# Perform the search
custom_search_result = grouped_sales[
    (grouped_sales['Dealer_Name'] == dealer_name_criteria) &
    (grouped_sales['Month'].isin(months_criteria)) &
    (grouped_sales['Vehicle_Line'] == vehicle_line_criteria)
]

# Display the search result
print(custom_search_result)

     Dealer_Name Brand  Year     Month Time  Vehicle_Line Vehicle_Type  \
4350      ZAPATA  FORD  2024   January  MTD  BRONCO SPORT   OUTFITTERS   
4333      ZAPATA  FORD  2024  February  MTD  BRONCO SPORT   OUTFITTERS   
4373      ZAPATA  FORD  2024     March  MTD  BRONCO SPORT   OUTFITTERS   

      Total_Sales  
4350           84  
4333           54  
4373           89  


In [14]:
# Concatenate the two DataFrames
pd_total_sales_q_m = pd.concat([grouped_sales, grouped_sales_quarterly], ignore_index=True)

# Display the resulting DataFrame
print("Number of records:", pd_total_sales_q_m.shape[0])


Number of records: 7271


In [15]:
import json

# Assuming 'grouped_sales_quarterly' contains the DataFrame with the sales data

# Define the documentation template
documentation_template = {
    "documentation": {
        "context": "This file contains the sales information of Ford of Mexico vehicle sales for 2024 at vehicle line level.",
        "terms": [
            {"name": "Dealer_Name", "definition": "Name of the car dealership"},
            {"name": "Brand", "definition": "Brand of the vehicle"},
            {"name": "Year", "definition": "Year of the sales data"},
            {"name": "Month", "definition": "Month of the sales data"},
            {"name": "Time", "definition": "Time period of the sales data, MTD is Month to date and QTY means Quarter"},
            {"name": "Vehicle_Line", "definition": "Type of vehicle line"},
            {"name": "Vehicle_Type", "definition": "Type of vehicle"}
        ]
    }
}

# Convert DataFrame to list of dictionaries
vehicle_sales_data = grouped_sales_quarterly.to_dict(orient='records')

# Combine documentation and vehicle_sales data
json_data = {
    **documentation_template,
    "vehicle_sales": vehicle_sales_data
}

# Write JSON data to file
file_location = '../json/llm_train_sales_data.json'
with open(file_location, 'w') as json_file:
    json.dump(json_data, json_file, indent=4)
